<a href="https://colab.research.google.com/github/tperellom/Projectes/blob/develop/Licitacion_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic libraries and packages**

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import re

**XML Data and dictionaries**

In [ ]:
# Open url and read the data

xml_url = urllib.request.urlopen('https://contrataciondelestado.es/sindicacion/sindicacion_643/licitacionesPerfilesContratanteCompleto3.atom')
xml_doc = xml_url.read()

In [ ]:
# Parse with BeautifulSoup
 
soup = BeautifulSoup(xml_doc,"lxml-xml")

In [ ]:
# Get the table with the contract types' codes

type_url = urllib.request.urlopen('https://contrataciondelestado.es/codice/cl/2.08/ContractCode-2.08.gc')
type_doc = type_url.read()
type_soup = BeautifulSoup(type_doc,"lxml-xml")
type_dict = {}
for row in type_soup.find_all('Row'):
  code = row.find('Value', {'ColumnRef': "code"}).text.strip()
  name = row.find('Value', {'ColumnRef': "nombre"}).text.strip()
  type_dict[name] = code

inv_type_dict = {v: k for k, v in type_dict.items()}

In [ ]:
# Get the table with the contract types' codes

subtype_works_url = urllib.request.urlopen('https://contrataciondelestado.es/codice/cl/1.04/WorksContractCode-1.04.gc') #'https://contrataciondelestado.es/codice/cl/1.04/CPV2007-1.04.gc'
subtype_works_doc = subtype_works_url.read()
subtype_works_soup = BeautifulSoup(subtype_works_doc,"lxml-xml")
subtype_works_dict = {}
for row in subtype_works_soup.find_all('Row'):
  code = row.find('Value', {'ColumnRef': "code"}).text.strip()
  name = row.find('Value', {'ColumnRef': "nombre"}).text.strip()
  name = name.replace('\n',' ').replace('\t','')
  subtype_works_dict[name] = code

inv_subtype_works_dict = {v: k for k, v in subtype_works_dict.items()}

In [ ]:
# Get the table with the contract types' codes

subtype_ser_url = urllib.request.urlopen('https://contrataciondelestado.es/codice/cl/1.04/ServiceContractCode-1.04.gc') #'https://contrataciondelestado.es/codice/cl/1.04/CPV2007-1.04.gc'
subtype_ser_doc = subtype_ser_url.read()
subtype_ser_soup = BeautifulSoup(subtype_ser_doc,"lxml-xml")
subtype_ser_dict = {}
for row in subtype_ser_soup.find_all('Row'):
  code = row.find('Value', {'ColumnRef': "code"}).text.strip()
  name = row.find('Value', {'ColumnRef': "nombre"}).text.strip()
  name = name.replace('\n',' ').replace('\t','')
  subtype_ser_dict[name] = code

inv_subtype_ser_dict = {v: k for k, v in subtype_ser_dict.items()}

In [ ]:
# Get the table with the NUTS' codes

nuts_url = urllib.request.urlopen('https://contrataciondelestado.es/codice/cl/2.0/NUTS-2009.gc')
nuts_doc = nuts_url.read()
nuts_soup = BeautifulSoup(nuts_doc,"lxml-xml")
nuts_dict = {}
for row in nuts_soup.find_all('Row'):
  code = row.find('Value', {'ColumnRef': "code"}).text.strip()
  name = row.find('Value', {'ColumnRef': "nombre"}).text.strip()
  nuts_dict[name] = code

spain_nuts_dict = {k:nuts_dict[k] for k in nuts_dict if re.match('(^ES[\w]*)', nuts_dict[k])}
inv_spain_nuts_dict = {v: k for k, v in spain_nuts_dict.items()}

In [ ]:
# Get the table with the contract status' codes

stat_url = urllib.request.urlopen('https://contrataciondelestado.es/codice/cl/2.04/SyndicationContractFolderStatusCode-2.04.gc')
stat_doc = stat_url.read()
stat_soup = BeautifulSoup(stat_doc,"lxml-xml")
stat_dict = {}
for row in stat_soup.find_all('Row'):
  code = row.find('Value', {'ColumnRef': "code"}).text.strip()
  name = row.find('Value', {'ColumnRef': "nombre"}).text.strip()
  stat_dict[name] = code

stat_dict = {k.upper(): v for k, v in stat_dict.items()}
inv_stat_dict = {v: k for k, v in stat_dict.items()}

**Selection**

In [ ]:
print(type_dict)
print(spain_nuts_dict)

{'Suministros': '1', 'Servicios': '2', 'Obras': '3', 'Gestión de Servicios Públicos': '21', 'Concesión de Servicios': '22', 'Concesión de Obras Públicas': '31', 'Concesión de Obras': '32', 'Colaboración entre el sector público y sector privado': '40', 'Administrativo especial': '7', 'Privado': '8', 'Patrimonial': '50'}
{'ESPAÑA': 'ES', 'NOROESTE': 'ES1', 'Galicia': 'ES11', 'A Coruña': 'ES111', 'Lugo': 'ES112', 'Ourense': 'ES113', 'Pontevedra': 'ES114', 'Principado de Asturias': 'ES12', 'Asturias': 'ES120', 'Cantabria': 'ES130', 'NORESTE': 'ES2', 'País Vasco': 'ES21', 'Álava': 'ES211', 'Guipúzcoa': 'ES212', 'Vizcaya': 'ES213', 'Comunidad Foral de Navarra': 'ES22', 'Navarra': 'ES220', 'La Rioja': 'ES230', 'Aragón': 'ES24', 'Huesca': 'ES241', 'Teruel': 'ES242', 'Zaragoza': 'ES243', 'COMUNIDAD DE MADRID': 'ES3', 'Comunidad de Madrid': 'ES30', 'Madrid': 'ES300', 'CENTRO (E)': 'ES4', 'Castilla y León': 'ES41', 'Ávila': 'ES411', 'Burgos': 'ES412', 'León': 'ES413', 'Palencia': 'ES414', 'Salama

In [ ]:
# Criteria:
location_criteria = [spain_nuts_dict['Mallorca'], 
                     spain_nuts_dict['Illes Balears'], 
                     spain_nuts_dict['Eivissa y Formentera'], 
                     spain_nuts_dict['Menorca']
                     ]

contract_type_criteria = [type_dict['Obras'], 
                          type_dict['Concesión de Obras'], 
                          type_dict['Concesión de Obras Públicas'],
                          type_dict['Gestión de Servicios Públicos'],
                          type_dict['Concesión de Servicios']
                          ]

In [ ]:
# Go across all the entries in the XML soup to find if any satisfies the criteria 

selected_entries = []
for entry in soup.find_all('entry'):

  # Contract Status
  contract_status = entry.find('cac-place-ext:ContractFolderStatus')
  id = contract_status.find('cbc:ContractFolderID').get_text()
  
  # Procurement project
  procurement_project = contract_status.find('cac:ProcurementProject')
  name = procurement_project.find('cbc:Name').get_text().strip()

  # Contract type
  contract_type = procurement_project.find('cbc:TypeCode').get_text()
  contract_subtype = procurement_project.find('cbc:SubTypeCode')
  contract_subtype_value = ''
  if contract_subtype:
    contract_subtype_value = contract_subtype.get_text()
    if contract_subtype_value in inv_subtype_works_dict.keys():
      contract_subtype_value = inv_subtype_works_dict[contract_subtype_value]

  # Location
  realized_location = procurement_project.find('cac:RealizedLocation')
  postal_code = realized_location.find('cbc:PostalZone')
  location_code = realized_location.find('cbc:CountrySubentityCode')
  postal_code_value = ''
  location_code_value = ''
  if location_code:
    location_code_value = location_code.get_text()
  if postal_code:
    postal_code_value = postal_code.get_text()

  # Check that the location and the contract type match with the criteria.
  # If so, register the entry
  if location_code_value in location_criteria and contract_type in contract_type_criteria:
    # or re.match('^07[\d]{3}', postal_code_value):

    # Status
    status = contract_status.find('cbc-place-ext:ContractFolderStatusCode').get_text()

    # Budget
    budget = procurement_project.find('cac:BudgetAmount')
    estimated_budget = budget.find('cbc:EstimatedOverallContractAmount')
    estimated_budget_value = ''
    if estimated_budget:
      estimated_budget_value = estimated_budget.get_text() 
      estimated_budget_value = '{:,}'.format(float(estimated_budget_value)) + ' ' + estimated_budget.attrs['currencyID']
  
    # Duration
    duration = procurement_project.find('cac:PlannedPeriod')
    estimated_duration = duration.find('cbc:DurationMeasure')
    estimated_duration_value = ''
    if estimated_duration:
      estimated_duration_value = estimated_duration.get_text() + ' ' + estimated_duration.attrs['unitCode']
    else:
      beginning = duration.find('cbc:StartDate')
      ending = duration.find('cbc:EndDate')
      if beginning and ending:
        estimated_duration_value = beginning.get_text() + ' to ' + ending.get_text()

    # Entity
    entity_name = contract_status.find('cac:PartyName').get_text().strip()

    # Tendering process
    tender = contract_status.find('cac:TenderingProcess')
    tender_deadline = tender.find('cac:TenderSubmissionDeadlinePeriod')
    deadline = ''
    if tender_deadline:
      if tender_deadline.find('cbc:EndDate'):
        deadline = tender_deadline.find('cbc:EndDate').get_text()

    # Links
    legal_doc = contract_status.find('cac:LegalDocumentReference')
    if legal_doc:
      legal_doc = legal_doc.find('cac:Attachment')
    legal_doc_value = ''
    if legal_doc:
      legal_doc_value = legal_doc.find('cbc:URI').get_text()
    
    tech_doc = contract_status.find('cac:TechnicalDocumentReference')
    tech_doc_value = ''
    if tech_doc:
      if tech_doc.find('cac:Attachment'):
        tech_doc_value = tech_doc.find('cac:Attachment').find('cbc:URI').get_text()

    selected_entry = {'ID': id, 
              'Name': name, 
              'Entity': entity_name,
              'Contract type': inv_type_dict[contract_type], 
              'Contract subtype': contract_subtype_value,
              'Status': inv_stat_dict[status],
              'Location': inv_spain_nuts_dict[location_code_value], 
              'Postal code': postal_code_value, 
              'Proposal deadline': deadline, 
              'Estimated budget': estimated_budget_value, 
              'Estimated duration': estimated_duration_value,
              'Legal document link': legal_doc_value,
              'Technical document link': tech_doc_value
              }
    
    selected_entries.append(selected_entry)

In [ ]:
selected_entries

[{'Contract subtype': 'Construcción',
  'Contract type': 'Obras',
  'Entity': "Institut Balear de l'Habitatge",
  'Estimated budget': '5,547,615.36 EUR',
  'Estimated duration': '18 MON',
  'ID': '46-2020',
  'Legal document link': 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=a4da5db2-d24e-41be-9bc4-8e044deef70b',
  'Location': 'Mallorca',
  'Name': 'Obras de ejecución de 43 viviendas de protección pública y aparcamiento en la c/ Brotad, de Palma (Mallorca)',
  'Postal code': '07007',
  'Proposal deadline': '2021-02-02',
  'Status': 'EN PLAZO',
  'Technical document link': 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=b0788b46-8a0d-4b48-8862-a79298fa3f34'}]

**Mailing selected entries**

In [ ]:
import smtplib
from email.mime.text import MIMEText

In [ ]:
# Write dictionaries from the selected entries as text in order to mail them:

### Plain text ###
# total_text = ''
# text = ''
# for entries in selected_entries:
#   for k, v in entries.items():
#     text = text + k + ': ' + v + '\n'
#   total_text = total_text + '\n\n' + text
#   text = ''

# text_type = 'plain'


### HTML message: ###
total_text = ''
text = ''
for entries in selected_entries:
  for k, v in entries.items():
    text = text + '<b>' + k + '</b>: ' + v + '<br/>'
  total_text = total_text + '<p>' + text + '<br/></p>'
  text = ''

total_text = '''
<h1 style="font-family:helvetica">Licitaciones</h1>
<body style="line-height:1.8; font-family:helvetica">''' + total_text + '''</body>'''

text_type = 'html'

# Set up the message for the mailing API
message = MIMEText(total_text, text_type)

message['Subject'] = 'Test Subject'
message['From'] = "no-reply <34d7a26d34-20009d@inbox.mailtrap.io>"
message['To'] = "Test User <roxegid446@ofdyn.com>"

# Send the mail
with smtplib.SMTP("smtp.mailtrap.io", 2525) as server:
    server.login("decdd53a13076f", "fe4c52996cc485")
    server.sendmail(message['From'], message['To'], message.as_string())